# Setup Code for VM related items
## !!!!When Saving, please ping revision so its saved in revision history!!!
Probs need to run anything under this section before code


In [4]:
!pip install alpaca-trade-api

Pandas Technical Analysis (Pandas TA) is an easy to use library that leverages the Pandas library with more than 120 Indicators and Utility functions. 

INFO HERE : https://github.com/twopirllc/pandas-ta#installation

USEFUL EXAMPLE: https://github.com/twopirllc/pandas-ta/blob/master/examples/AIExample.ipynb

In [5]:
!pip install --upgrade ta

Requirement already up-to-date: ta in /usr/local/lib/python3.7/dist-packages (0.7.0)


In [33]:
!pip install yfinance

     |████████████████████████████████| 5.5MB 19.1MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=1fe686bb94ec8c8b66058d485d014355c1b506949fe802f3098c7c7e3b738cfb
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


#Code
code cells are unfortunately not linked together and a such all code for program has to go on one cell

Alpaca Secret : Y01uwQ4EFeZQzanLj1UVs7NqyCAHvbwo6UyrxqFw

Alpaca Key : PK2K1WCVGJBQ0T4BKDIW

Yahoo Finance Key : 75ee96eb2fmsha6dbdb2310728dfp1301d2jsnbf21f7a615c1

In [40]:
import alpaca_trade_api as alpacaapi
import yfinance as yf, pandas as pd, shutil, os, time, glob
import matplotlib.pyplot as plt
import pytz as pytz
import requests
import json
import numpy as np
import datetime
import time
from datetime import timedelta
from ta import add_all_ta_features
from ta.utils import dropna
from ta.momentum import RSIIndicator
from ta.trend import MACD, SMAIndicator, EMAIndicator
from ta.momentum import StochasticOscillator
from ta.volume import OnBalanceVolumeIndicator
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from google.colab import drive


#drive.mount("/content/drive/")
api = alpacaapi.REST("PKHCRUMLZWP36N6UIYOQ", "nHyyYJQ7RRpAyqJyE1NkPZsYkBft3Xj46OyDSdzI", "https://paper-api.alpaca.markets")

################
# getStockData #
################

# https://alpaca.markets/docs/api-documentation/api-v2/market-data/bars/
# Check the above link for specific usage if you need to use it in your function
def getStockData(tick, length, frame):
  barset = api.get_barset(tick, frame , limit=length).df
  return barset

#Example usage
#  This will get the info for Tesla for the last 100 minutes
#  getStockData("TSLA", 100, "minute")


#####################
# getStockDataMulti #
#####################

# getStockDataMulti creates a list of dataframes containing
# time-aggregated price and volume data 
def getStockDataMulti(ticks, length, frame):
  dataframes = []
  for tick in ticks:
    df = getStockData(tick, length, frame)
    dataframes.append({ tick: df })
  return dataframes

# Example usage
#   stocks = ["TSLA", "AAPL", "SNAP"]
#   data = getStockDataMulti(stocks, 5, "minute")
#   print(data)

######################
# getTrendingTickers #
######################

# getTrendingTickers creates a list of the current top 20
# trending stocks from Yahoo Finance
def getTrendingTickers():
  # build request
  url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/market/get-trending-tickers"
  querystring = {"region":"US"}
  headers = {
      'x-rapidapi-key': "75ee96eb2fmsha6dbdb2310728dfp1301d2jsnbf21f7a615c1",
      'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
  }
  # execute request
  response = requests.request("GET", url, headers=headers, params=querystring)
  raw_data = json.loads(response.text)['finance']['result']
  # extract symbols
  symbols = []
  for quote in raw_data[0]['quotes']:
    symbols.append(quote['symbol'])
  return symbols

# Example usage
#   symbols = getTrendingTickers()
#   print(symbols)



#############
# sellLimit #
#############

# A sell limit is submitted as a gtc(Good 'Til Cancelled) time in force which 
# keeps the sell limit order open until it is fulfilled or cancelled.
# You must pass in the current price of the tick as the third argument when the
# sell limit is placed. This is so the 10% price increase can be calculated.
def sellLimit(tick, quantity, current_price):
    try:
        api.submit_order(
            symbol=symbol,
            qty=quantity,
            side='sell',
            type='limit',
            time_in_force='gtc',
            limit_price = current_price * 1.1
        )
    # if an excpetion is thrown, prints exception message and returns 0 to 
    # indicate a failed sell limit placement
    except Exception as e:
        print(e)
        return 0

    # sellLimit function returns 1 if the order is placed succesfully
    else:
        return 1
        
# Example usage
#   result = sellLimit('AAPL', 1, 445)
#   print("Result of sell limit placement is 0 for fail, 1 for success: ", result)




def sellStocks(stockName, quantity, sellType, ToF):
  return api.submit_order(
      symbol=stockName,
      qty=quantity,
      side='sell',
      type=sellType, # market, limit, stop, stop_limit, or trailing_stop
      time_in_force=ToF #day, gtc, opg, cls, ioc, fok.
  )

# Example usage
stockName = 'AAPL'
quantity = 5
sellType = 'market'
ToF = 'gtc'
#stockOrder = sellStocks(stockName, quantity, sellType, ToF)

#print(stockOrder)










#Ben

#####################
#     buyStocks     #
#####################
def buyStocks(stockName, quantity, buyType, ToF):
  return api.submit_order(
      symbol=stockName,
      qty=quantity,
      side='buy',
      type=buyType, # market, limit, stop, stop_limit, or trailing_stop
      time_in_force=ToF #day, gtc, opg, cls, ioc, fok.
  )

# Example usage
stockName = 'AAPL'
quantity = 5
buyType = 'market'
ToF = 'gtc'
#stockOrder = buyStocks(stockName, quantity, buyType, ToF)

#print(stockOrder)








#Bryan

def stopLimit(tick, quantity, current_price):
    try:
        api.submit_order(
            symbol=symbol,
            qty=quantity,
            side='stop',
            type='stop_limit',
            time_in_force='gtc',
            limit_price = current_price * 0.7
        )
    # if an excpetion is thrown, prints exception message and returns 0 to 
    # indicate a failed sell limit placement
    except Exception as e:
        print(e)
        return 0

    # stopLimit function returns 1 if the order is placed succesfully
    else:
        return 1



##############################
# Golden/Death cross checker #
##############################
def maCross(ema, sma, openEprevEMAMA, prevSMA):
  if (ema == sma) and (prevSMA < prevEMA):
    return 1 # golden cross
  elif (ema == sma) and (prevSMA > prevEMA):
    return 2 # death cross
  return 0 # no cross


#determines if the AI should buy right now
def isBuy(ticker, duration, timeFrame):
  df = getStockData(ticker, duration, timeFrame)

  #Drop Null Values
  df = dropna(df)


  #Declare Indicators
  rsi = RSIIndicator(close=df[ticker]["close"], window=20, fillna=False)
  macd = MACD(close=df[ticker]["close"], window_slow=20, window_fast = 16, window_sign= 7, fillna = False)
  stoch = StochasticOscillator(high=df[ticker]["high"],low=df[ticker]["low"],close=df[ticker]["close"], window=14, smooth_window=3, fillna=False)
  onvol = OnBalanceVolumeIndicator(close=df[ticker]["close"], volume=df[ticker]["volume"])
  ema = EMAIndicator(close=df[ticker]["close"], window=200, fillna=False)
  sma = SMAIndicator(close=df[ticker]["close"], window=50, fillna=False)
  prevEMA = EMAIndicator(close=df.head(-1)[ticker]["close"], window=200, fillna=False)
  prevSMA = SMAIndicator(close=df.head(-1)[ticker]["close"], window=50, fillna=False)

  # check if there is a golden/death cross
  # cross = maCross(ema, sma, prevEMA, prevSMA)

  # Add Technical Analysis indicators to dataframes
  df['rsi'] = rsi.rsi()
  df['macd'] = macd.macd_diff()
  df['stoch'] = stoch.stoch_signal()
  df['onbalvol'] = onvol.on_balance_volume()
  df['ema'] = ema.ema_indicator()
  df['sma'] = sma.sma_indicator()

  # get todays entry and run filter on it
  curr = df.tail(1)
  curr = curr.loc[(curr["rsi"]>40) & (curr["rsi"]<55)]
  curr = curr.loc[(curr["macd"]> -1) & (curr["macd"]< 1)]
  curr = curr.loc[(curr['stoch']> 20) & (curr['stoch']< 40)]

  # if not empty after filter run, check close price is within 25% below or above ema, then check onbalance volume for slope
  if not curr.empty:
    if (curr["close"] > (curr["close"] - curr["ema"]*.25) or curr["close"] < (curr["close"] + curr["ema"]*.25)):
      sday = df.tail(7)
      head = sday.head(1) 
      tail = sday.tail(1)
      dayone = (head.index.values[0] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")
      dayseven = (tail.index.values[0] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")
      slope = (int(tail["onbalvol"].values) - int(head["onbalvol"].values)) / (dayseven - dayone)
      # positive returns true, negative returns false
      if slope > 0:
        return True
      else:
        return False
    #if price not within 25% of ema, return false
    else:
      return False
  #if dataframe is null, return false
  else:
    return False


#determines if the AI should buy right now
def isSell(ticker, duration, timeFrame):
  df = getStockData(ticker, duration, timeFrame)

  #Drop Null Values
  df = dropna(df)


  #Declare Indicators
  rsi = RSIIndicator(close=df[ticker]["close"], window=20, fillna=False)
  macd = MACD(close=df[ticker]["close"], window_slow=20, window_fast = 16, window_sign= 7, fillna = False)
  stoch = StochasticOscillator(high=df[ticker]["high"],low=df[ticker]["low"],close=df[ticker]["close"], window=14, smooth_window=3, fillna=False)
  onvol = OnBalanceVolumeIndicator(close=df[ticker]["close"], volume=df[ticker]["volume"])
  ema = EMAIndicator(close=df[ticker]["close"], window=200, fillna=False)
  sma = SMAIndicator(close=df[ticker]["close"], window=50, fillna=False)

  # Add Technical Analysis indicators to dataframes
  df['rsi'] = rsi.rsi()
  df['macd'] = macd.macd_diff()
  df['stoch'] = stoch.stoch_signal()
  df['onbalvol'] = onvol.on_balance_volume()
  df['ema'] = ema.ema_indicator()
  df['sma'] = sma.sma_indicator()

  # get todays entry and run filter on it
  curr = df.tail(1)
  curr = curr.loc[(curr["rsi"]>65) & (curr["rsi"]<90)]
  curr = curr.loc[(curr["macd"]> -1) & (curr["macd"]< 0)]
  curr = curr.loc[(curr['stoch']> 70) & (curr['stoch']< 100)]

  # if not empty after filter run, check close price is within 25% below or above ema, then check onbalance volume for slope
  if not curr.empty():
    if (curr["close"] > (curr["close"] - curr["ema"]*.25) or curr["close"] < (curr["close"] + curr["ema"]*.25)):
      sday = df.tail(7)
      head = sday.head(1) 
      tail = sday.tail(1)
      dayone = (head.index.values[0] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")
      dayseven = (tail.index.values[0] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")
      slope = (int(tail["onbalvol"].values) - int(head["onbalvol"].values)) / (dayseven - dayone)
      # negative slope returns true, positive returns false
      if slope < 0:
        return True
      else:
        return False
    #if price not within 25% of ema, return false
    else:
      return False
  #if dataframe is null, return false
  else:
    return False




#################
#LOG REGRESSION ALGO#
#################
def Log_Model(tfone, tftwo, tfthree):
    # Check that the amount of tickers isn't more than 2000
  tickers = getTrendingTickers();
  
  print("The amount of stocks chosen to observe: " + str(len(tickers)))
  # These two lines remove the Stocks folder and then recreate it in order to remove old stocks. Make sure you have created a Stocks Folder the first time you run this.
  shutil.rmtree("./stocks")
  os.mkdir("./stocks/")

  # Holds the amount of API calls we executed
  Amount_of_API_Calls = 0

  # This while loop is reponsible for storing the historical data for each ticker in our list. Note that yahoo finance sometimes incurs json.decode errors and because of this we are sleeping for 2
  # seconds after each iteration, also if a call fails we are going to try to execute it again.
  # Also, do not make more than 2,000 calls per hour or 48,000 calls per day or Yahoo Finance may block your IP. The clause "(Amount_of_API_Calls < 1800)" below will stop the loop from making
  # too many calls to the yfinance API.
  # Prepare for this loop to take some time. It is pausing for 2 seconds after importing each stock.

  # Used to make sure we don't waste too many API calls on one Stock ticker that could be having issues
  Stock_Failure = 0
  Stocks_Not_Imported = 0

  # Used to iterate through our list of tickers
  i=0
  while (i < len(tickers)) and (Amount_of_API_Calls < 1800):
      try:
          stock = tickers[i]  # Gets the current stock ticker
          temp = yf.Ticker(str(stock))
          Hist_data = temp.history(period="max")  # Tells yfinance what kind of data we want about this stock (In this example, all of the historical data)
          Hist_data.to_csv("./stocks/"+stock+".csv")  # Saves the historical data in csv format for further processing later
          time.sleep(2)  # Pauses the loop for two seconds so we don't cause issues with Yahoo Finance's backend operations
          Amount_of_API_Calls += 1 
          Stock_Failure = 0
          i += 1  # Iteration to the next ticker
          print("Importing stock data:" + str(i))
      except ValueError:
          print("Yahoo Finance Backend Error, Attempting to Fix")  # An error occured on Yahoo Finance's backend. We will attempt to retreive the data again
          if Stock_Failure > 5:  # Move on to the next ticker if the current ticker fails more than 5 times
              i+=1
              Stocks_Not_Imported += 1
          Amount_of_API_Calls += 1
          Stock_Failure += 1
  print("The amount of stocks we successfully imported: " + str(i - Stocks_Not_Imported))


  # These two lines remove the Stocks folder and then recreate it in order to remove old stocks. Make sure you have created a Stocks Folder the first time you run this.
  # Get the Y values
  list_files = (glob.glob("./stocks/*.csv")) # Creates a list of all csv filenames in the stocks folder
  for interval in list_files:
      Stock_Name = ((os.path.basename(interval)).split(".csv")[0])
      data = pd.read_csv(interval)
      dropna(data)
      try: 
        data = add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume")
      except:
        continue

      data = data.iloc[100:]
      close_prices = data['Close'].tolist()
      firsttf_Obs = []
      secondtf_Obs = []
      thirdtf_Obs = []
      x = 0
      while x < (len(data)):
          if x < (len(data)-tfone):
              if ((close_prices[x+1] + close_prices[x+2] + close_prices[x+3] + close_prices[x+4] + close_prices[x+5])/5) > close_prices[x]:
                  firsttf_Obs.append(1)
              else:
                  firsttf_Obs.append(0)
          else:
              firsttf_Obs.append(0)
          x+=1
      y = 0
      while y < (len(data)):
          if y < (len(data)-tftwo):
              secondtf = 0
              y2 = 0
              while y2 < tftwo:
                  secondtf = secondtf + close_prices[y+y2]
                  y2 += 1
              if (secondtf/tftwo) > close_prices[y]:
                  secondtf_Obs.append(1)
              else:
                  secondtf_Obs.append(0)
          else:
              secondtf_Obs.append(0)
          y+=1
      z = 0
      while z < (len(data)):
          if z < (len(data)-tfthree):
              thirdtf = 0
              z2 = 0
              while z2 < tfthree:
                  thirdtf = thirdtf + close_prices[z+z2]
                  z2 += 1
              if (thirdtf/tfthree) > close_prices[z]:
                  thirdtf_Obs.append(1)
              else:
                  thirdtf_Obs.append(0)
          else:
              thirdtf_Obs.append(0)
          z+=1  
      data['ftf'] = firsttf_Obs
      data['stf'] = secondtf_Obs
      data['ttf'] = thirdtf_Obs
      print(data['ftf'])
    
      data.to_csv("./stocks/"+Stock_Name+".csv")
      print("Data for " + Stock_Name + " has been substantiated with technical features.")

  Hold_Results = []
  list_files2 = (glob.glob("./stocks/*.csv")) # Creates a list of all csv filenames in the stocks folder
  for interval2 in list_files2:
      Stock_Name = ((os.path.basename(interval2)).split(".csv")[0])
      data = pd.read_csv(interval2,index_col=0)
      data = data.replace([np.inf, -np.inf], np.nan)
      data = data.fillna(0)
      try:
        dependents = [data["ftf"].to_list(), data["stf"].to_list(), data["ttf"].to_list()]
      except:
        continue
      data = data.drop(['ftf', 'stf', 'ttf', 'Date', 'Open', 'High', 'Low', 'Close'], axis = 1)
      print(data)
      scaler = StandardScaler()
      data = scaler.fit_transform(data)  # Standardize our data set
      Hold_Results_Section = []
      p = 0
      for dep in dependents:
          x_train, x_test, y_train, y_test =\
          train_test_split(data, dep, test_size=0.2, random_state=0)
          model = LogisticRegression(solver='liblinear', C=0.05, multi_class='ovr',random_state=0)
          model.fit(x_train, y_train)
          y_pred = model.predict(x_test)  # To get the predicted values
          conf = confusion_matrix(y_test, y_pred)
          if p == 0:
              Hold_Results.append([Stock_Name, tfone, model.score(x_train, y_train),model.score(x_test, y_test),conf[0,0],conf[0,1],conf[1,0],conf[1,1]])
          if p == 1:
              Hold_Results.append([Stock_Name, tftwo, model.score(x_train, y_train),model.score(x_test, y_test),conf[0,0],conf[0,1],conf[1,0],conf[1,1]])
          if p == 2:
              Hold_Results.append([Stock_Name, tfthree, model.score(x_train, y_train),model.score(x_test, y_test),conf[0,0],conf[0,1],conf[1,0],conf[1,1]])
          p+=1
      print("Model complete for " + Stock_Name)
  df = pd.DataFrame(Hold_Results, columns=['Stock', 'Observation Period', 'modacctrain', 'modacctest', 'True Positives','False Positives',
  'False Negative','True Negative'])
  df = df[df['modacctest'] > .7]
  df.to_csv("./stocks/Model_Outcome.csv", index = False)
  return df






#This gets the current time in the chicago timezone and converts it to hours:minutes:seconds
now = pytz.timezone("America/Chicago")
now = datetime.datetime.now(now)
#Creates the end market time. So we have to start the program at 8:45am and it will run until 2:45pm at which point the while loop should quit
endMarket = now + timedelta(hours=8)

##############
# Main Loop #
##############

pd.set_option('display.max_columns', None)
  # get trending tickers
account = api.get_account()
while now != endMarket:

  now = pytz.timezone("America/Chicago")
  now = datetime.datetime.now(now)

  #Get current holdings
  portfolio = api.list_positions()
  print(portfolio)

  for tick in portfolio:
    #check price vs purchase prices,  if price 10% greater set stop loss
      if tick.unrealized_plpc > .10:
        if isSell(tick.symbol, 672, "15Min"):
          # run is_sell
          sellStocks(tick.symbol, int(tick.qty), "limit", "cls")
  # if false, continue holding, if true sell if profit margin met
  print("Portfolio Up to date")
  
  
  
  #at this point portfolio should be up to date
  
  
  
  # run ML algo against trending tickets
  # check all entries for all instances where accuracy over 70% for starters
  # returns 7 day stocks, 14 day stocks, 21 day stocks, Observation period column is ints so should be able to compare when buying
  # Chris - limit this to Run every 30 minutes, yahoo caps querys to 2000 an hour and 48000 a day i ran it like 6 times in an hour and it was fine
  # i think the day cap might be more strict
  model_Data = Log_Model(7, 14, 21) 
  print("log_models Finished")


  # add to watch list
  # run isbuy for all watch list tickers on specified timelength where accuracy over 75%
  # for example, if acurracy is 76% percent for 5 day, run isbuy on the 5day timeframe to see if we can buy
  # buy if true returned
  # remove from watch list
  bought_symbols= []

  # Chris - This needs to be modified to work with the new dataframe returned by the log algo
  for tick.Stock in model_Data:
    if isBuy(tick.Stock, 672, "15Min"):
      tData = getStockData(tick, 1, "minute")
      quantity = (account.buying_power*.1) / tData.tail(1).close
      if quantity == 0:
        continue
      buyStocks(tick.Stock, quantity, "limit", "day")
      bought_symbols.append(tick.Stock)
 
  time.sleep(300)
 
  
  





# outside of while loop, once it quits

# clear watch list

[]
Portfolio Up to date
The amount of stocks chosen to observe: 20
Importing stock data:1
Importing stock data:2
Importing stock data:3
Importing stock data:4
Importing stock data:5
Importing stock data:6
Importing stock data:7
Importing stock data:8
Importing stock data:9
Importing stock data:10
Importing stock data:11
Importing stock data:12
Importing stock data:13
Importing stock data:14
Importing stock data:15
Importing stock data:16
Importing stock data:17
Importing stock data:18
Importing stock data:19
Importing stock data:20
The amount of stocks we successfully imported: 20


/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


100    1
101    1
102    0
103    0
104    0
      ..
463    0
464    0
465    0
466    0
467    0
Name: ftf, Length: 368, dtype: int64
Data for JFIN has been substantiated with technical features.


/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


100     1
101     1
102     1
103     1
104     1
       ..
6325    0
6326    0
6327    0
6328    0
6329    0
Name: ftf, Length: 6230, dtype: int64
Data for IDRA has been substantiated with technical features.


...

/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


100      1
101      1
102      1
103      1
104      1
        ..
12635    0
12636    0
12637    0
12638    0
12639    0
Name: ftf, Length: 12540, dtype: int64
Data for ^IXIC has been substantiated with technical features.


/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


Streaming output truncated to the last 5000 lines.
103           2740.000           2640.00                 2520.000   
104           2780.000           2720.00                 2540.000   
...                ...               ...                      ...   
6325             4.635              4.83                    5.260   
6326             4.615              4.83                    5.260   
6327             4.615              4.83                    5.245   
6328             4.615              4.83                    5.245   
6329             4.950              4.83                    5.245   

      trend_visual_ichimoku_b  trend_aroon_up  trend_aroon_down  \
100                  2440.000            16.0              72.0   
101                  2440.000            12.0              68.0   
102                  2440.000             8.0              64.0   
103                  2440.000           100.0              60.0   
104                  2440.000            96.0              56

AttributeError: ignored